In [17]:
import gspread
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime

In [2]:
service_account = gspread.service_account(filename='./config/expense-tracker-358105-199d116b0a6d.json')
workbook = service_account.open('badminton_tracking')
worksheet = workbook.get_worksheet(0)

In [3]:
df = pd.DataFrame(worksheet.get_all_records()).drop(["Timestamp", "result"], axis=1)

df.columns = ["date", "team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2", "points_team_1", "points_team_2", "venue"]

df['winner'] = np.where(df.points_team_1 > df.points_team_2, 'team_1', 'team_2')
df['margin'] = abs(df.points_team_1 - df.points_team_2)
df['total_points_per_game'] = df["points_team_1"] + df["points_team_2"]

df = df.applymap(lambda x: f'{x}'.lower().strip() if isinstance(x, str) else x)

In [35]:
expense_data = pd.DataFrame(workbook.worksheet('expense_tracker').get_all_records()).set_index('date').reset_index()

In [36]:
expense_data

,date,amount,paid_by
0,2023-03-05,380,ajay
1,2023-03-06,200,raghotham


In [100]:
def fn(x):
    return list(set(', '.join([', '.join(x[i]) for i in range(4)]).split(', ')))

players_on_date = df[['date', 'team_1_player_1', 'team_1_player_2', 'team_2_player_1', 'team_2_player_2']].copy()

players_on_date['date'] = players_on_date['date'].apply(lambda x: f'{datetime.strptime(x, "%m/%d/%Y").date()}')

players_on_date = players_on_date.groupby("date").agg({
    'team_1_player_1': 'unique',
    'team_1_player_2': 'unique',
    'team_2_player_1': 'unique',
    'team_2_player_2': 'unique'
})

players_on_date['players'] = players_on_date[['team_1_player_1', 'team_1_player_2', 'team_2_player_1', 'team_2_player_2']].apply(fn, axis=1)
players_on_date = players_on_date[['players']]
players_on_date['number_of_players'] = players_on_date['players'].apply(lambda x: len(x))

In [104]:
player_share = pd.merge(players_on_date.explode('players'), expense_data, how="inner", on="date")
player_share['share'] = player_share['amount'] / player_share['number_of_players']

In [105]:
player_share

,date,players,number_of_players,amount,paid_by,share
0,2023-03-05,ajay,5,380,ajay,76.0
1,2023-03-05,vinay,5,380,ajay,76.0
2,2023-03-05,raghotham,5,380,ajay,76.0
3,2023-03-05,srinidhi,5,380,ajay,76.0
4,2023-03-05,prateek,5,380,ajay,76.0
5,2023-03-06,ajay,4,200,raghotham,50.0
6,2023-03-06,vinay,4,200,raghotham,50.0
7,2023-03-06,raghotham,4,200,raghotham,50.0
8,2023-03-06,sameer,4,200,raghotham,50.0


In [107]:
# balances = 
player_share.groupby('players').agg({
    'date': 'count',
    'share': 'sum'
})

,date,share
players,,
ajay,2,126.0
prateek,1,76.0
raghotham,2,126.0
sameer,1,50.0
srinidhi,1,76.0
vinay,2,126.0
